#  Journey demonstration

With JuPedSim, directing agents towards exits and ensuring a smooth evacuation from the simulation area is straightforward and versatile. 
There might be scenarios where it's vital to navigate agents along various paths, thus creating diverse evacuation situations. 
Let's explore different routing strategies of agents using a simple geometric space - a corner.

JuPedSim manages routing by geometrically triangulating the simulation area. Without user-defined routing strategies, agents, for example, in a corner simulation, naturally move towards the inner edge of the corner. Look at this visualization where the given direction of each agent is shown by a red line. You'll observe all red lines lead towards the exit along the inner edge of the corner. While it seems logical, this path isn’t always optimal and could result in a bottleneck, thereby slowing down the evacuation process.

![](./shortest_path.gif)


In [ ]:
from shapely import GeometryCollection, Polygon
import pathlib
import pandas as pd
import numpy as np
import jupedsim as jps
import sqlite3 
import plotly.express as px
import plotly.graph_objects as go
import pedpy
from pedpy.column_identifier import ID_COL, FRAME_COL
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print(f"JuPedSim: {jps.__version__}\nPedPy: {pedpy.__version__}")

## Preparing the Simulation: Geometry and Routing Instructions

Let's start by setting up a basic polygon. This will serve as our main simulation area where agents will be distributed. 
Additionally, we'll mark an exit area using another polygon. When agents enter this exit area, they're deemed to have safely evacuated and will be removed from the ongoing simulation.

Next, we'll introduce an initial target for the agents: a sizable circular area (known as a switch). After the simulation kickstarts, agents will first head towards this circle. Once they enter the circle, they'll be directed to one of three distinct waypoints, set diagonally along the curve of the corner.

For the simulation's onset, all agents will be positioned inside a rectangular zone at the corner's base.

In [ ]:
simulation_polygon = Polygon([(-3, 6), (-3, -3), (15, -3), (15, 0), (0, 0), (0, 6)])
exit_polygon = [(-3, 6), (0, 6), (0, 5.5), (-3, 5.5)]
switch_point = (2, -1.5)
waypoints = [
    (-0.5, -0.5),
    (-1.5, -1),
    (-2.5, -1.5),
]
distance = 1
distribution_polygon = Polygon([[14.8, -0.2], [3.8, -0.2], [3.8, -2.8], [14.8, -2.8]])

# define geometry objections for simulation with jupedsim and analysis with pedpy
geometry = jps.geometry_from_shapely(GeometryCollection(simulation_polygon))
walkable_area = pedpy.WalkableArea(simulation_polygon)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
pedpy.plot_walkable_area(walkable_area=walkable_area,axes=ax);
x, y = distribution_polygon.exterior.xy
plt.fill(x, y, alpha=0.1) 
plt.plot(x, y, color='black')
centroid = distribution_polygon.centroid
plt.text(centroid.x, centroid.y, "Distribution Polygon", ha='center', va='center', fontsize=12)
ax.plot(switch_point[0], switch_point[1], "bo")
ax.annotate(f"Switch", (switch_point[0], switch_point[1]), textcoords="offset points", xytext=(-10,-15), ha='center')
for idx, waypoint in enumerate(waypoints): 
    ax.plot(waypoint[0], waypoint[1], "ro")
    ax.annotate(f"WP {idx+1}", (waypoint[0], waypoint[1]), textcoords="offset points", xytext=(10,-15), ha='center')

In [ ]:
num_agents = 100
positions = jps.distribute_by_number(
    polygon=distribution_polygon,
    number_of_agents=num_agents,
    distance_to_agents=0.4,
    seed=12,
    distance_to_polygon=0.2,
)

## Exploring Transition Strategies

All agents initially set their course towards the switch_point. After reaching it, they navigate towards intermediate goals (waypoints) before making their way to the final exit. The challenge lies in deciding which waypoint each agent should target next.

Let's explore three unique methods to determine these transition strategies:

1. **Direct Path Strategy**: Here, every agent simply aims for the first waypoint, mirroring a shortest path algorithm.
2. **Balanced Load Strategy**: Agents are directed towards the least occupied waypoint, ensuring a more balanced distribution.
3. **Round Robin Strategy**: Waypoints are sequentially assigned to agents, rotating through each in turn.
---------

###  Direct Path Strategy

In [ ]:
def shortest_path(simulation:jps.Simulation, switch_id, waypoint_ids, exit_id):
    """Build a journey with fixed transitions for a given simulation."""

    journey = jps.JourneyDescription([switch_id, *waypoint_ids, exit_id])
    # switch ---> 1st waypoint
    journey.set_transition_for_stage(
        switch_id, jps.Transition.create_fixed_transition(waypoint_ids[0])
    )
    # 1st waypoint ---> exit
    journey.set_transition_for_stage(
        waypoint_ids[0], jps.Transition.create_fixed_transition(exit_id)
    )

    journey_id = simulation.add_journey(journey)
    return journey_id

### Balanced Load Strategy

In [ ]:
def least_targeted(simulation:jps.Simulation, switch_id, waypoint_ids, exit_id):
    """Build a journey with least targeted transitions for a given simulation."""

    journey = jps.JourneyDescription([switch_id, *waypoint_ids, exit_id])
    # switch ---> least targeted waypoint
    journey.set_transition_for_stage(
        switch_id, jps.Transition.create_least_targeted_transition(waypoint_ids)
    )
    # from all waypoints ---> exit
    for waypoint_id in waypoint_ids:
        journey.set_transition_for_stage(
            waypoint_id, jps.Transition.create_fixed_transition(exit_id)
        )

    journey_id = simulation.add_journey(journey)
    return journey_id

### Round Robin Strategy

In [ ]:
def round_robin(simulation:jps.Simulation, switch_id, waypoint_ids, exit_id):
    """Build a journey with least round-robin transitions for a given simulation."""

    journey = jps.JourneyDescription([switch_id, *waypoint_ids, exit_id])
    # switch ---> 1st waypoint with weight 1
    # switch ---> 2s waypoint with weight 2
    # switch ---> 3th waypoint with weight 4
    journey.set_transition_for_stage(
        switch_id,
        jps.Transition.create_round_robin_transition(
            [
                (waypoint_ids[0], 1),
                (waypoint_ids[1], 2),
                (waypoint_ids[2], 4),
            ]
        ),
    )
    # from all waypoints ---> exit
    for waypoint_id in waypoint_ids:
        journey.set_transition_for_stage(
            waypoint_id, jps.Transition.create_fixed_transition(exit_id)
    )

    journey_id = simulation.add_journey(journey)
    return journey_id

In [ ]:
scenarios = [
    shortest_path,
    least_targeted,
    round_robin,
]

## Executing the Simulation

With all components in place, we're set to initiate the simulation.
For this demonstration, the trajectories will be recorded in an sqlite database.

In [ ]:
def run_scenario_simulation(scenario, agent_parameters, positions, geometry):
    """Runs a simulation for a given scenario using the provided simulation object, agent parameters, and positions."""
    filename = f"{scenario.__name__}.sqlite"
    simulation = jps.Simulation(
        model=jps.VelocityModelParameters(a_ped=2.6, d_ped=0.1, d_wall=0.05),
        geometry=geometry,
        trajectory_writer=jps.SqliteTrajectoryWriter(
            output_file=pathlib.Path(filename)
        ),
    )
    exit_id = simulation.add_exit_stage(exit_polygon)
    switch_id = simulation.add_waypoint_stage(switch_point, 3)
    waypoint_ids = [
        simulation.add_waypoint_stage(waypoint, distance)
        for waypoint in waypoints
    ]
    agent_parameters.stage_id = switch_id
    journey_id = scenario(simulation, switch_id, waypoint_ids, exit_id)
    agent_parameters.journey_id = journey_id
    for new_pos in positions:
        agent_parameters.position = new_pos
        simulation.add_agent(agent_parameters)

    while simulation.agent_count() > 0:
        simulation.iterate()

    print(
        f"Simulation completed after {simulation.iteration_count()} iterations.\n"
        f">> Output: {filename}"
    )

    return filename

In [ ]:
def init_agent_parameters():
    """Initialise the agent parameters with predefined values."""
    agent_parameters = jps.VelocityModelAgentParameters()
    agent_parameters.orientation = (-1.0, 0.0)
    agent_parameters.time_gap = 1
    agent_parameters.tau = 0.5
    agent_parameters.v0 = 1.2
    agent_parameters.radius = 0.2
    return agent_parameters

In [ ]:
for scenario in scenarios:
    print(f"Running simulation with <{scenario.__name__}> strategy ...")
    agent_parameters = init_agent_parameters()
    filename = run_scenario_simulation(scenario, agent_parameters, positions, geometry)

In [ ]:
import matplotlib.pyplot as plt
DUMMY_SPEED = -1000
def read_sqlite_file(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )


def speed_to_color(speed, min_speed, max_speed):
    """Map a speed value to a color using a colormap."""
    normalized_speed = (speed - min_speed) / (max_speed - min_speed)
    r, g, b = plt.cm.jet_r(normalized_speed)[:3]
    return f"rgba({r*255:.0f}, {g*255:.0f}, {b*255:.0f}, 0.5)"

def get_line_color(disk_color):

    r, g, b, _ = [int(float(val)) for val in disk_color[5:-2].split(",")]
    brightness = (r * 299 + g * 587 + b * 114) / 1000
    return "black" if brightness > 127 else "white"  

def create_orientation_line(row, line_length=0.2, color='black'):
    end_x = row["x"] + line_length * row["ox"]
    end_y = row["y"] + line_length * row["oy"]

    orientation_line = go.layout.Shape(
        type="line",
        x0=row["x"],
        y0=row["y"],
        x1=end_x,
        y1=end_y,
        line=dict(color=color, width=3),
    )
    return orientation_line

def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces

def get_colormap(frame_data, max_speed):
    """Utilize scatter plots with varying colors for each agent instead of individual shapes.

    This trace is only to incorporate a colorbar in the plot.
    """
    scatter_trace = go.Scatter(
        x=frame_data["x"],
        y=frame_data["y"],
        mode="markers",
        marker=dict(
            size=frame_data["radius"] * 2,
            color=frame_data["speed"],
            colorscale="Jet_r",
            colorbar=dict(title="Speed [m/s]"),
            cmin=0,
            cmax=max_speed,
        ),
        text=frame_data["speed"],
        showlegend=False,
        hoverinfo="none",
    )

    return [scatter_trace]

def get_shapes_for_frame(frame_data, min_speed, max_speed):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        if row["speed"] == DUMMY_SPEED:
            dummy_trace = go.Scatter(
                x=[row["x"]],
                y=[row["y"]],
                mode="markers",
                marker=dict(size=1, opacity=0),
                hoverinfo="none",
                showlegend=False,
            )
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                dummy_trace, create_orientation_line(row, color="rgba(255,255,255,0)")
            )
        color = speed_to_color(row["speed"], min_speed, max_speed)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace, create_orientation_line(row,color=get_line_color(color))
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    arrows =  [res[2] for res in results]
    return shapes, hover_traces, arrows

def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_arrows,
    initial_hover_trace,
    initial_scatter_trace,
    geometry_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
):
    """Creates a Plotly figure with animation capabilities.

    Returns:
        go.Figure: A Plotly figure with animation capabilities.
    """

    minx, miny, maxx, maxy = area_bounds
    title = f"<b>Number of Agents: {initial_agent_count}</b>"
    fig = go.Figure(
        data=geometry_traces
        + initial_scatter_trace
        #+ hover_traces
        + initial_hover_trace,
        frames=frames,
        layout=go.Layout(
            shapes=initial_shapes + initial_arrows, title=title, title_x=0.5
        ),
    )
    fig.update_layout(
        updatemenus=[_get_animation_controls()],
        sliders=[_get_slider_controls(steps)],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )

    return fig


def _get_animation_controls():
    """Returns the animation control buttons for the figure."""
    return {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": 100, "redraw": True},
                        "fromcurrent": True,
                    },
                ],
                "label": "Play",
                "method": "animate",
            },
            {
                "args": [
                    [None],
                    {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0},
                    },
                ],
                "label": "Pause",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }


def _get_slider_controls(steps):
    """Returns the slider controls for the figure."""
    return {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 100, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": steps,
    }

def animate(
    data: pedpy.TrajectoryData, area: pedpy.WalkableArea, *, every_nth_frame=50
):
    data_df = pedpy.compute_individual_speed(traj_data=data, frame_step=5)
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = 0.2
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    max_agents = data_df.groupby("frame").size().max()
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hover_trace, initial_arrows = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed
    )
    color_map_trace = get_colormap(
        initial_frame_data, max_speed
    )

    for frame_num in selected_frames[1:]:
        frame_data, agent_count = _get_processed_frame_data(
            data_df, frame_num, max_agents
        )
        shapes, hover_traces, arrows = get_shapes_for_frame(
            frame_data, min_speed, max_speed
        )
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes + arrows,
                title=f"<b>Number of Agents: {agent_count}</b>",
                title_x=0.5,
            ),
        )
        frames.append(frame)

        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_arrows,
        initial_hover_trace,
        color_map_trace,
        geometry_traces,
        frames,
        steps,
        area.bounds,
        width=800,
        height=800,
    )


def _get_processed_frame_data(data_df, frame_num, max_agents):
    """Process frame data and ensure it matches the maximum agent count."""
    frame_data = data_df[data_df["frame"] == frame_num]
    agent_count = len(frame_data)
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": DUMMY_SPEED}
    while len(frame_data) < max_agents:
        dummy_df = pd.DataFrame([dummy_agent_data])
        frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)
    return frame_data, agent_count

## Visualizing the Trajectories

To visualize trajectories, we'll pull simulation data from the SQLite database and then employ a helper function to depict the agent movements. For subsequent analyses, we'll organize these trajectory files within a dictionary for easy access.

In [ ]:
agent_trajectories = {}
scenario_name = scenarios[0].__name__
print(scenario_name)
agent_trajectories[scenario_name], walkable_area = read_sqlite_file(f"{scenario_name}.sqlite")
animate(agent_trajectories[scenario_name], walkable_area)

In [ ]:
scenario_name = scenarios[1].__name__
agent_trajectories[scenario_name], walkable_area = read_sqlite_file(f"{scenario_name}.sqlite")
print(scenario_name)
animate(agent_trajectories[scenario_name], walkable_area)

In [ ]:
scenario_name = scenarios[2].__name__
agent_trajectories[scenario_name], walkable_area = read_sqlite_file(f"{scenario_name}.sqlite")
print(scenario_name)
animate(agent_trajectories[scenario_name], walkable_area)

## Analysis of the results

With three distinct evacuation simulations completed, it's time to dive into the outcomes. Let's start by visualizing the trajectories. This will give us an initial insight into the variations among the scenarios:

In [ ]:
fig, axes  = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
for name, ax in zip(agent_trajectories, axes):    
    pedpy.plot_trajectories(traj=agent_trajectories[name], walkable_area=walkable_area,axes=ax)
    ax.set_title(name)

## Compute density and speed for all three simulations

In [ ]:
# densities = {}
# for name in agent_trajectories:
#     densities[name] = pedpy.compute_individual_voronoi_polygons(
#         traj_data=agent_trajectories[name],
#         walkable_area=walkable_area,
#     )

In [ ]:
# speeds = {}
# for name in agent_trajectories:
#     speeds[name] = pedpy.compute_individual_speed(
#         traj_data=agent_trajectories[name],
#         frame_step=5,
#         speed_calculation=pedpy.SpeedCalculation.BORDER_SINGLE_SIDED,
#     )

## Calculate profiles

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")
# grid_size = 1 # this is big, and hould be smaller, but I can't wait any longer!
# density_profiles = {}
# speed_profiles = {}
# for name in agent_trajectories:
#     density_profiles[name], speed_profiles[name] = pedpy.compute_profiles(
#         individual_voronoi_speed_data=pd.merge(
#             densities[name],
#             speeds[name],
#             on=[ID_COL, FRAME_COL],
#         ),
#         walkable_area=walkable_area.polygon,
#         grid_size=grid_size,
#         speed_method=pedpy.SpeedMethod.ARITHMETIC,
#     )

In [ ]:
# fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 10))
# for name, ax in zip(agent_trajectories, axes): 
#     cm = pedpy.plot_profiles(
#         walkable_area=walkable_area,
#         profiles=density_profiles[name],
#         axes=ax,
#         label="$\\rho$ / 1/$m^2$",
#         vmax=2.,
#         title=name,
#     )   
# fig.tight_layout(pad=2)

In [ ]:
from scipy import stats
from typing import Tuple
import numpy.typing as npt
def calculate_density_average_classic(
    bounds: Tuple[float, float, float, float],
    dx: float,
    nframes: int,
    X: npt.NDArray[np.float64],
    Y: npt.NDArray[np.float64],
) -> npt.NDArray[np.float64]: 
    """Calculate classical method

    Density = mean_time(N/A_i)
    """
    geominX, geominY, geomaxX, geomaxY = bounds
    xbins = np.arange(geominX, geomaxX + dx, dx)
    ybins = np.arange(geominY, geomaxY + dx, dx)
    area = dx * dx
    ret = stats.binned_statistic_2d(
        X,
        Y,
        None,
        "count",
        bins=[xbins, ybins],
    )
    return np.array(np.nan_to_num(ret.statistic.T)) / nframes / area

In [ ]:
from plotly.subplots import make_subplots
def plot_classical_density_profile(data, walkable_area, name, dx, rho_max):
    vmax = rho_max
    geominX, geominY, geomaxX, geomaxY = walkable_area.bounds
    title = f"<b>{name}</b>"
    fig = make_subplots(rows=1, cols=1, subplot_titles=([title]))
    xbins = np.arange(geominX, geomaxX + dx, dx)
    ybins = np.arange(geominY, geomaxY + dx, dx)
    x, y = walkable_area.polygon.exterior.xy
    x = list(x)
    y = list(y)
    heatmap = go.Heatmap(
        x=xbins,
        y=ybins,
        z=data,
        zmin=0,
        zmax=rho_max,
        name=title,
        connectgaps=False,
        zsmooth='best',
        hovertemplate="%{z:.2f}<br> x: %{x:.2f}<br> y: %{y:.2f}",
        colorbar=dict(title="Density"),
        colorscale="Jet",
    )
    fig.add_trace(heatmap)
    #    Geometry walls
    line = go.Scatter(
        x=x,
        y=y,
        mode="lines",
        name="wall",
        showlegend=False,
        line=dict(
            width=3,
            color="white",
        ),
    )
    fig.add_trace(line)

    return fig

In [ ]:
dx=0.5
rho_max = -1
fig = make_subplots(rows=1, cols=3, subplot_titles=(list(agent_trajectories.keys())))
for count, name in enumerate(agent_trajectories):
    trajectories = agent_trajectories[name]
    data = calculate_density_average_classic(
        walkable_area.bounds,
        dx,
        nframes=trajectories.data["frame"].max(),
        X=trajectories.data["x"],
        Y=trajectories.data["y"],
    )
    rho_max = max(np.max(data), rho_max)
    ind_fig = plot_classical_density_profile(data, walkable_area, name, dx, rho_max)
    for trace in ind_fig.data:
        fig.add_trace(trace, row=1, col=count+1)
   
    fig.update_xaxes(title_text="X [m]", row=1, col=count+1)
    fig.update_yaxes(title_text="Y [m]", scaleanchor="x", scaleratio=1)
fig        

## Analyzing Evacuation Duration

To further understand our earlier observations, we compute the $N−t$ diagram, which shows when an agent crosses a designated measurement line. 
We position this line near the exit and evaluate the $N−t$ curves for all three simulations, subsequently determining the respective evacuation durations.

Note: It's essential to position the measurement line inside the simulation area, ensuring that agents **cross** it.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
measurement_line = pedpy.MeasurementLine([[-3,4],[0,4]])
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
colors = ["blue", "red", "green"]
for i, name in enumerate(agent_trajectories):
    nt, _ = pedpy.compute_n_t(
        traj_data=agent_trajectories[name],
        measurement_line=measurement_line,
    )
    ax = pedpy.plot_nt(nt=nt, color=colors[i]);
    ax.lines[-1].set_label(name)
    Time = np.max(nt['time'])
    print("Name: {:<20} Evacuation time: {:<15}".format(name, "{} seconds".format(Time)))
ax.legend()
plt.show()

## Findings and Summary

From the density profiles, it is evident that, in line with our expectations, the shortest path algorithm results in higher densities prior to reaching the corner. On the other hand, the round-robin algorithm appears to be more effective, distributing agents around the corner with greater efficiency.

## References & Further Exploration